# Quickstart

First, we need to install the AgentChat and Extension packages `pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"`

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Deinfe a model client, you can use other client that implements
# the 'ChatCompletionClient' interface

model_client = OpenAIChatCompletionClient(
    model="gpt-4.1-nano", # $0.20 / 1M tokens
    api_key=os.getenv("OPENAI_API_KEY")
)

# https://openai.com/api/pricing/

In [3]:
# Define a simple function tool that the agent can use.
# For this examplem we use a fake weather tool for demonstration purpose.

async def get_weather(city:str)->str:
    """
    Get the weather for a given city.
    """
    return f"The weather in {city} is 73 degrees and Sunny!"

In [4]:
from autogen_agentchat.agents import AssistantAgent
# Define a AssistantAgent with the model, tool, system message and reflection enabled
# The system message instructs the agent via natural language.

system_message = """
You are a helpful assistant.
"""

agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message=system_message,
    reflect_on_tool_use=True,
    model_client_stream=True
)

- `reflect_on_tool_use`: When enabled, the agent will analyze and learn from its tool usage patterns. The agent will generate additional reflections after using tools Helps the agent improve its tool usage over time. Can provide better context for future interactions. Must be explicitly set (no default value)

- `model_client_stream (bool)`: Controls whether the model client streams tokens as they're generated. When False: Waits for the complete response before returning.

In [5]:
from autogen_agentchat.ui import Console

# Run the agent and stream the messages to the console
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))
    # Close the connection to the model client
    await model_client.close()

In [6]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_CEoA37mOFgshOmVpGsbpDuNg', arguments='{"city": "New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny!', name='get_weather', call_id='call_CEoA37mOFgshOmVpGsbpDuNg', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is 73 degrees and sunny.


# Explaination

Let me break down the output you're seeing, which is a sequence of events in an agent's interaction. I'll explain each component and its role in the `autogen_agentchat` framework.

## **Event Flow Explanation**

### **1. TextMessage (user)**

**Plaintext:**

```
What is the weather in New York?
```

**What it is:** A message from the user to the agent

**Role:** Represents the initial user input

**Implementation:** Defined in `autogen_agentchat/messages.py` as a basic message type

---

### **2. ToolCallRequestEvent (weather\_agent)**

**Plaintext:**

```
[FunctionCall(id='call_CEoA37mOFgshOmVpGsbpDuNg', arguments='{"city": "New York"}', name='get_weather')]
```

**What it is:** The agent's request to execute a tool

**Key Components:**

* **id:** Unique identifier for this tool call
* **name:** The tool being called (`get_weather`)
* **arguments:** The parameters being passed (`{"city": "New York"}`)

**Implementation:** Defined in `autogen_agentchat/messages.py` as an event signaling tool usage

---

### **3. ToolCallExecutionEvent (weather\_agent)**

**Plaintext:**

```
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny!', name='get_weather', call_id='call_CEoA37mOFgshOmVpGsbpDuNg', is_error=False)]
```

**What it is:** The result of the tool execution

**Key Components:**

* **content:** The actual result from the tool
* **name:** The tool that was executed
* **call\_id:** Matches the ID from the `ToolCallRequestEvent`
* **is\_error:** Indicates if there was an error

**Implementation:** Defined in `autogen_agentchat/messages.py` as an event signaling tool execution results

---

### **4. ModelClientStreamingChunkEvent (weather\_agent)**

**Plaintext:**

```
The weather in New York is 73 degrees and sunny.
```

**What it is:** The final, formatted response being streamed to the user

**Role:** Provides real-time streaming of the model's response

**Implementation:** Defined in `autogen_agentchat/messages.py` for streaming text chunks

---

## **How It Works Together**

1. The user sends a question about the weather.
2. The agent (`weather_agent`) processes this and determines it needs to call the `get_weather` tool.
3. A **`ToolCallRequestEvent`** is generated with the tool call details.
4. The tool executes and produces a **`ToolCallExecutionEvent`** with the result.
5. The agent formats the result and streams it back as a **`ModelClientStreamingChunkEvent`**.

---

## **Key Components in the Codebase**

### **Message Types (in `messages.py`)**

* **TextMessage:** For simple text messages

* **ToolCallRequestEvent:** For tool call requests

* **ToolCallExecutionEvent:** For tool execution results

* **ModelClientStreamingChunkEvent:** For streaming text chunks

### **Agent Implementation**

* **AssistantAgent** (in `_assistant_agent.py`): Handles the logic for processing messages and generating tool calls

* **CodeExecutorAgent** (in `_code_executor_agent.py`): Handles code execution if needed

### **Tool Integration**

* Tools are defined as callable objects that the agent can invoke
* The agent uses the tool's schema to generate proper function calls

---

### **Summary**

This **event-based architecture** allows for **flexible and observable agent interactions**, making it easier to **debug** and **extend** the system's capabilities.